In [86]:
import json
import pandas as pd
import numpy as np
import os
import re

pd.options.display.max_columns = 999

In [55]:
# load in wiki data
DSB_folder = os.getenv('DSB_PATH')

with open(f"{DSB_folder}8_ETL/wikipedia-movies.json", mode='r') as file:
    wiki_movies_raw = json.load(file)

In [56]:
len(wiki_movies_raw)

7311

In [57]:
# First 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [58]:
# Last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [59]:
# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [60]:
# load in kaggle data
kaggle_metadata = pd.read_csv(f'{DSB_folder}8_ETL/movies_metadata.csv', low_memory=False)

ratings = pd.read_csv(f'{DSB_folder}8_ETL/ratings.csv')

In [61]:
kaggle_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
13463,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,38789,tt0373915,sv,I Manegen med Glenn Killing: Live från Berns,Glenn Killing with guests. One of the most suc...,0.002328,/c7Ye3NHDf31WfbowOlABkZOl88t.jpg,[],"[{'iso_3166_1': 'SE', 'name': 'Sweden'}]",1995-09-30,0.0,220.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",Released,NaN,I Manegen med Glenn Killing: Live från Berns,False,0.0,0.0
23760,False,NaN,0,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",NaN,99324,tt0034558,en,Busses Roar,A sergeant (Richard Travis) saves the day when...,0.142022,/7oBX8D00la2ADSJE9fbEvaFE7X7.jpg,"[{'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1942-08-18,0.0,58.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"BYE-BYE, SPY! HERE COME THE MARINES!...What ac...",Busses Roar,False,8.0,1.0
15789,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,8886,tt1008017,de,Palermo Shooting,After the wild life-style of a famous young Ge...,0.595492,/cyHGXb8H2cosR4jUPRhv07SfN9d.jpg,"[{'name': 'Rectangle Productions', 'id': 2700}...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2008-11-20,0.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Palermo Shooting,False,5.1,14.0
16768,False,NaN,10400000,"[{'id': 10752, 'name': 'War'}]",NaN,51200,tt1587729,en,포화 속으로,A civil war broke out in the 1950s between Nor...,3.106814,/1v4XKKlJJKpUVhaDKy5XWV3A9pG.jpg,"[{'name': 'Taewon Entertainment', 'id': 3965},...","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2010-06-16,20967660.0,120.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,71: Into the Fire,False,7.0,38.0
38874,False,NaN,0,[],NaN,394244,tt0167688,en,Plastic Utopia,In a world where moral corruption and prosperi...,0.215128,/wLnjMCKH9Pv4Qe4oPDegRXE0Oci.jpg,[],[],1997-10-04,0.0,98.0,[],Released,What the hell is going on?,Plastic Utopia,False,8.0,1.0


In [62]:
ratings.sample(5)

,userId,movieId,rating,timestamp
17369949,180256,3771,4.0,1042381629
15165723,157810,1114,3.0,879005805
22475706,233314,27773,5.0,1128482164
4666824,47925,594,5.0,1069453842
20602450,214137,46972,0.5,1485470937


In [63]:
# messy raw df
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

#wiki_movies_df.head()

In [64]:
# get col names
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [65]:
# get only movies, no tv etc
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

len(wiki_movies)

7076

In [66]:
# raw df of movies
wiki_movies_df = pd.DataFrame(wiki_movies)

wiki_movies_df.head(3)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,Narrated by,Music by,Cinematography,Edited by,Productioncompany,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Written by,Genre,Theme music composer,Country of origin,Original language(s),Producer(s),Editor(s),Production company(s),Original network,Original release,Productioncompanies,Executive producer(s),Production location(s),Distributor,Picture format,Audio format,Voices of,Followed by,Composer(s),Created by,Preceded by,Adaptation by,Suggested by,Traditional,Mandarin,Released,Recorded,Venue,Length,Label,Director,Producer,Also known as,Animation by,Color process,Simplified,French,Screen story by,Hangul,Revised Romanization,McCune–Reischauer,Original title,Japanese,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay","[Cliff Eidelman, Yello]",Oliver Wood,Michael Tronick,Silver Pictures,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,English,$20 million,$21.4 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Maurice Jarre,Mark Plummer,Howard E. Smith,Avenue Pictures,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,English,$6 million,$2.7 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Charles Gross,Roger Deakins,"[John Bloom, Lois Freeman-Fox]","[Carolco Pictures, IndieProd Company]",TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,"[English, Lao]",$35 million,"$57,718,089",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
def clean_movie(movie):
    
    movie = dict(movie) #create a non-destructive copy
    
    alt_titles = {}
    
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        
        if key in movie:
            alt_titles[key] = movie[key]
            # extract alt title
            movie.pop(key)
    
    
    if len(alt_titles) > 0:
        # set alt title to foreign lang title
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            # put data in column with new name
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [68]:
# iterate over the json
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [69]:
# new clean df
wiki_movies_df = pd.DataFrame(clean_movies)

# get col name list
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'McCune–Reischauer',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [70]:
# get just imdb movies

# in the raw string: () incl, tt match 2 lc t's, \d match digit, {7} match last thing (digit) 7 times
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

print(len(wiki_movies_df))

# get rid of dups by imdb ID
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)

print(len(wiki_movies_df))

wiki_movies_df.head(3)

7076
7033


,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,Country,Language,Budget,Box office,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),Genre,Original language(s),Original network,Executive producer(s),Production location(s),Picture format,Audio format,Voices of,Followed by,Created by,Preceded by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,English,$20 million,$21.4 million,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,English,$6 million,$2.7 million,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,"[English, Lao]",$35 million,"$57,718,089",Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005


In [76]:
# track nulls per column

# list comprehension
#[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]
 
# add over 90% not null condition
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]


['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']

In [77]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

# select for mostly not null columns
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [78]:
wiki_movies_df.head(3)

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,Language,Budget,Box office,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,English,$20 million,$21.4 million,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,English,$6 million,$2.7 million,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,"[English, Lao]",$35 million,"$57,718,089",Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005


In [79]:
# parse the data--needs to be in correct format! (numeric etc)

wiki_movies_df.dtypes

url                      object
year                      int64
imdb_link                object
title                    object
Based on                 object
Starring                 object
Cinematography           object
Release date             object
Running time             object
Country                  object
Language                 object
Budget                   object
Box office               object
Director                 object
Distributor              object
Editor(s)                object
Composer(s)              object
Producer(s)              object
Production company(s)    object
Writer(s)                object
imdb_id                  object
dtype: object

In [81]:
# need to change Box office, Budget, Running time to numeric; Release date to date

# make box_office df for testing
box_office = wiki_movies_df['Box office'].dropna()

In [82]:
# regular expressions only work on strings, so we need to make sue all values are strings

# func to check for non-strings
def is_not_a_string(x):
    return type(x) != str

# use map to apply
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [85]:
# lambda is better here as a one use func
box_office[box_office.map(lambda x: type(x) != str)]

Series([], Name: Box office, dtype: object)

In [84]:
# we have some lists in the data :(

# join the data if it is a list and has space between it
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# now use regular expressions to fix the weird data (ie, 1.5 million instead of 1,500,000)
## regular expressions are strings of characters used as a search pattern
## they can test if strings are in a specific format or contain substring in specific format
### can also extract info and discard rest, or perform complex replacement of substrings

# special characters in regex:
##  \d - match any digit 0-9
##  \D - match any non-digit character
##  \w - match a word character (letter, digit, underscore)
##  \W - match any non-word character (anything else incl spaces)
##  \s - match any whitespace character (incl space, tab, \n)
##  \S - match any non-whitespace character

# character sets: defined by []
## special character sets:
###   "[a-z]" - match any lc letter
###   "[A-Z]" - match any uc letter
###   "[0-9]" - match any digit
###   "[a-zA-Z]" - match any letter
###   "[a-zA-Z0-9]" - match any alphanumeric character
####    ranges can be constrained like "[A-E]", "[1-3]" etc
####    we can use special characters inside ranges: "[a-zA-Z\d]" = "[a-zA-Z0-9]"
####    can use just characters to pick multiple unranged: ex "[mb]illion" for million/billion

# specify a character to NOT be included with ^

# . is a wildcard and will match any single character except line break
## to specificy an ACTUAL period, use \.

# \ is the escape character, turning anything after it into a regular character

# special counting characters: specify how many times character should show up
##  * means character can repeat any number of times, incl 0
###     ex: "ca*t" match "cat", "caaat", "fiCTion"
##  + means character can repeat any number of times more than 0
###     ex: "ca*t" match "cat", "caaat"
## {} for an exact number of times--can also be a range
###     ex: "ca{3}t" match "caaat", "ca{3,5}" match "caaat", "caaaat", "caaaaat"
##  ? for 1 or 0 times
###     ex: "ca?t" match "cat", "ct", "fiCTion"

# | for OR
## ex: "cat|dog" match "cat", "dog"

# string boundaries
## ^ for start of string
###     ex: "^cat" match "cat", "catatonic", NOT "tomcat"
##  $ for end of string
###     ex: "cat$" match "cat", "tomcat", NOT "catatonic"

# add a structure to the search
##  ex:  "\d{3}-\d{3}-\d{4}" would match any ###-###-#### phone number

# capture groups () can extract only parts of specified string:
##  ex: "(\d{3})-(\d{3})-(\d{4})" gets only digits of phone number, no hyphens

# non-capture groups uses (?:) for structure but does not capture ino
## most useful for match and replace
###   ex: "(\d{3})-(?:\d{3})-(?:\d{4})" would return only area code

# negative lookahead groups (?!) are non-capture groups that check ahead and make sure string doesn't exist after match
## useful for checking to make sure we don't have similar but longer data being picked up
### ex: "(\d{3})-(\d{3})-(\d{4})(?!\d)" will pick up phone number 333-333-4444 but not user ID 333-333-44444

In [87]:
# build regex expression for budget
# dollar sign + one or more digits + optional decimal point + optional decimals + optional space(s) +  m/b illion
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [88]:
# get number of data points matching format (case insensitive)
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3896

In [89]:
# dollar sign + 1-3 digits + , followed by 3 digits (possibly repeated more than once)
form_two = r'\$\d{1,3}(?:,\d{3})+'

In [90]:
# get number of data points matching format (case insensitive)
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1544

In [91]:
# find data points not in either form
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)

matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

# does not match either with ~ negative operator
box_office[~matches_form_one & ~matches_form_two]

34                         US$ 4,212,828
79                              $335.000
110                   $4.35-4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1070                          35,254,617
1147    $ 407,618 (U.S.) (sub-total) [1]
1446                        $ 11,829,959
1480                          £3 million
1611                            $520.000
1865                        ¥1.1 billion
2032                                 N/A
2091                                $309
2130               US$ 171.8 million [9]
2257                   US$ 3,395,581 [1]
2263            $ 1,223,034 ( domestic )
2347                            $282.175
2638            $ 104,883 (US sub-total)
2665         926,423 admissions (France)
2697      $ 1.7 million (US) (sub-total)
2823                            $414.000
2924                            $621.000
3088           $

In [93]:
# fix pattern matches

# get rid of space between dollar sign and number by adding \s* after dollar sign
form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

# some values have period as thousands separator, change with [,\.]
## only want it to apply to ones NOT formatted as [mb]illion (ie, 1.234 million)
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

# some values given as range--starts with $ and ends with -
## replace with just $
### to replace dash we need hyphen, em dash, en dash
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

# million misspelled as 'millon'
## make second i optional with ?
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'


In [94]:
# extract and convert box office values

# extract() rturns dataframe where every column is the data that matches the capture group(s)
## our f string specifies match form_one or form_two
box_office.str.extract(f'({form_one}|{form_two})')

,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7070,$19.4 million
7071,$41.9 million
7072,$76.1 million
7073,$38.4 million


In [95]:
# function to convert to standard numeric

def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [96]:
# extract and apply function
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

C:\Users\charl\AppData\Local\Temp\ipykernel_7344\2562454313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)


In [98]:
# drop old column
wiki_movies_df.drop('Box office', axis=1, inplace=True)

C:\Users\charl\AppData\Local\Temp\ipykernel_7344\1444928999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies_df.drop('Box office', axis=1, inplace=True)
